In [1]:
gpu_id = 3
device = f"cuda:{gpu_id}"

In [2]:
import sys, os
from tqdm import tqdm
import torch
sys.path.append("./road")
from submission import model_submission, get_test_outputs, select_bboxes, format_results
from models import get_model_detection
from config import model_path as model_dir
from config import submission_path 

In [3]:
batch_size = 2
num_workers = 1
mode = "class"

model_path = f"{model_dir}/models/class/resnet101_lr_0.005_annealing_6/epoch_23"

In [4]:
# Extract results
model = get_model_detection(5).to(device)
model.load_state_dict(torch.load(model_path))
results = get_test_outputs(model)

  0%|          | 0/2631 [00:00<?, ?it/s]/home/tristan/anaconda3/envs/detectron/lib/python3.8/site-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/opt/conda/conda-bld/pytorch_1587428207430/work/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)
100%|██████████| 2631/2631 [03:08<00:00, 13.93it/s]


In [5]:
# Filter results
thr = .8
maxbbox = 5
filtered_results = {}
lbls = []
for key, anno in tqdm(results.items()):
    filtered_results[key] = select_bboxes(anno, thr, maxbbox)
    lbls.extend(filtered_results[key]["labels"].cpu().numpy().tolist())
set(lbls)

100%|██████████| 2631/2631 [00:00<00:00, 5615.90it/s]


{1, 2, 3, 4}

In [7]:
text_results = format_results(filtered_results, sep=",")

,


In [ ]:
name = f"resnet101"
path = submission_path + name
with open(path, "w") as f:
    f.write(text_results)